In [1]:
from pandas import DataFrame
from data201 import db_connection, df_query

In [2]:
conn = db_connection('sheql_wh.ini')
cursor = conn.cursor()

In [3]:
def display_results(cursor):
    """
    If there are results in the cursor from a call to a
    stored procedure, display the results in a dataframe. 
    """
    columns = result.description
    
    if columns == None:
        return 
    
    else:
        column_names = [column_info[0] 
                        for column_info in columns]

        rows = cursor.fetchall()
    
        if len(rows) > 0:
            df = DataFrame(rows)
            df.columns = column_names
            
            display(df)
        
        else:
            return

# KPIs

### Get Total Schools 

In [4]:
cursor.execute('DROP PROCEDURE IF EXISTS get_total_schools')

cursor.execute(
    """
    CREATE PROCEDURE get_total_schools()
    BEGIN
        SELECT COUNT(*) AS total_schools FROM school_dim;
    END
    """
)

In [5]:
cursor.callproc('get_total_schools')

for result in cursor.stored_results():
    display_results(result)

,total_schools
0,10


## Total Number of Students

In [6]:
cursor.execute('DROP PROCEDURE IF EXISTS total_students')

cursor.execute(
    """
    CREATE PROCEDURE total_students()
    BEGIN
        SELECT COUNT(*) AS total_students FROM student_dim;
    END
    """
)

In [7]:
cursor.callproc('total_students')

for result in cursor.stored_results():
    display_results(result)

,total_students
0,3000


## Abscence Rate

In [18]:
cursor.execute('DROP PROCEDURE IF EXISTS abscence_rate')

cursor.execute(
    """
    CREATE PROCEDURE abscence_rate()
    BEGIN
        SELECT
        ROUND(
            SUM(CASE WHEN status = 'absent' THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            2
        ) AS absence_rate_percent
        FROM student_attendance_fact;
    END
    """
)

In [19]:
cursor.callproc('abscence_rate')

for result in cursor.stored_results():
    display_results(result)

,absence_rate_percent
0,3.57


In [20]:
cursor.close()
conn.close()